In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D, ReLU, Flatten, BatchNormalization

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
samples_per_class = 5000
no_of_classes = 2

In [ ]:
noise_df = pd.read_csv("/content/gdrive/MyDrive/GW_SOP/Data/Final_Merged_Noise_Reduced_No_Abs.csv", header=None)
noise = noise_df.values.astype(float)


data_BBH_df = pd.read_csv("/content/gdrive/MyDrive/GW_SOP/Data/Final_BBH_Merged_Noise_Signal_Reduced_No_ABS.csv", header=None)
data_BBH = data_BBH_df.values.astype(float)

In [ ]:
X = np.concatenate((noise, data_BBH), axis=0)

print(len(noise_df.index))
print(X)
print(X.shape)

In [ ]:
# Alternate way of creating y for the dataset
 
y = [int(i/samples_per_class) for i in range(samples_per_class*no_of_classes)]
y = tf.keras.utils.to_categorical(y)
print(y)

In [ ]:
X *= 1e19
print(X)

# X *= 1e18
# print(X)

In [ ]:
X = np.expand_dims(X, axis=-1)
print(X.shape)
print(X)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2) #, random_state=1

In [ ]:
print(X_train.shape, y_train.shape)

In [10]:
model = Sequential()

model.add(Conv1D(128, 3, input_shape = (16384,1)))
model.add(MaxPool1D(4, 4))
model.add(Dropout(0.45))
model.add(ReLU())
model.add(Conv1D(16, 3, activation = 'sigmoid'))
model.add(MaxPool1D(4, 4))
model.add(Dropout(0.1))
model.add(Conv1D(8, 3))
model.add(MaxPool1D(4, 4))
model.add(BatchNormalization())
model.add(Dropout(0.05))
model.add(ReLU())
model.add(Conv1D(128, 3, activation = 'sigmoid'))
model.add(MaxPool1D(4, 4))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_16 (Conv1D)           (None, 16382, 128)        512       
_________________________________________________________________
dropout_16 (Dropout)         (None, 16382, 128)        0         
_________________________________________________________________
re_lu_8 (ReLU)               (None, 16382, 128)        0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 16380, 16)         6160      
_________________________________________________________________
dropout_17 (Dropout)         (None, 16380, 16)         0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 16378, 8)          392       
_________________________________________________________________
batch_normalization_4 (Batch (None, 16378, 8)         

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_val, y_val))

In [ ]:
acc = [0.] + history.history['accuracy']

loss = history.history['loss']

plt.figure(figsize=(13, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training Accuracy')
plt.xlabel('epoch')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.legend(loc='upper right')
plt.ylabel('Categorical Cross Entropy')
plt.ylim([0,2])
plt.title('Training Loss')
plt.xlabel('epoch')
plt.show()